#### 컨볼루션 신경망(CNN : Convolutional Neural Network)

In [ ]:
# CNN은 이미지뿐만 아니라 텍스트나 음성등 다양한 분야의 데이터 처리에 사용되고 있다.
# 이번 장에서는 CNN 사용법중 가장 기초가 되는 이미지 데이터를 다루는 방법을 알아보자.

# 4장에 나오는 보스턴 주택가격 데이터세트에서는 주택의 가격을 예측하기 위한 주택당 방의 개수, 재산세율, 범죄율 같은
# 특징(feature)들이 있었다. 5장에서 나온 와인 데이터세트에서도 와인의 종류나 품질을 예측하기 위한 당도, 알코올 도수
# 같은 특징들이 데이터로 존재했다.
# 이에 비해 FashionMNIST 같은 이미지 데이터에서는 어떤 특징을 찾을 수 있을까?
# 이미지 데이터에서는 연구자가 스스로 특징을 찾아야 한다. 
# 이미지 특징 추출(Feature Extraction) 기법중 하나인 컨볼루션 연산은 각 픽셀을 본래 픽셀과 그 주변 픽셀의 조합으로
# 대체하는 동작이다. 이때 연산에 사용되는 작은 행렬을 filter 또는 kernel이라고 한다.
# 컨볼루션 연산은 우리말로 합성곱이라고 한다. 원본 이미지의 각 픽셀을 포함한 주변 픽셀과 필터의 모든 픽셀을 각각
# 곱연산을 하고 그 결과들을 모두 합해서 새로운 이미지에 넣어주기 때문에 합성곱이라는 이름을 붙인 것 같다.  

In [1]:
# 간단한 컨볼루션 필터 몇 가지를 소개해보자.
# 3*3 크기의 작은 필터는 필터를 구성하는 셀마다 각각 다른 숫자를 넣어서 원본 이미지의 수직선검출, 수평선검출,
# 흐림(blur), 날카로움(sharpen) 이미지 효과등을 줄 수 있다. 
# 그런데 이런 필터들은 경험적 지식을 통해 직접 손으로 값을 넣어준 결과인 것이다. 이러한 수작업으로 설계한 특징에는
# 다음 세가지의 문제점이 있다. 첫째, 적용하고자 하는 분야에 대한 전문적 지식이 필요하다. 둘째, 수작업으로 특징을 
# 설계하는 것은 시간과 비용이 많이 드는 작업이다. 세째, 한분야(예를 들어 이미지)에서 효과적인 특징을 다른 분야
# (예를 들어 음성)에 적요하기가 어렵다.

# 딥러닝 기반의 컨볼루션 연산은 이런 문제들을 모두 해결했다. 즉 컨볼루션 신경망은 특징을 검출하는 필터를 사람이 
# 수작업으로 설계하는 것이 아니라 네트워크 스스로 특징을 추출하는 필터를 자동으로 생성한다. 
# 학습을 계속하면 네트워크를 구성하는 각 뉴런들은 입력한 데이터에 대해 특정 패턴을 잘 추출할 수 있도록 적응하게 된다.
# 그러면 컨볼루션 신경망은 어떻게 특징을 자동으로 추출하는 것일까? 컨볼루션 신경망의 주요 레이어를 하나씩 살펴보면서
# 그 방법을 알아보기로 하자.

In [ ]:
# 주요 레이어 정리

# 지금까지 배운 레이어는 Dense레이어와 Flatten레이어 두 종류였다. Dense레이어는 신경망에서 사용하는 가장 기본적인
# 레이어로 각 뉴런이 서로 완전하게 연결되어 있기 때문에 완전연결레이어(Fully Connected Layer)라고 한다. 
# Flatten레이어는 다차원의 이미지를 1차원으로 평평하게 바꿔주는 단순한 레이어이다.

# 이미지 분류에 사용되는 컨볼루션 신경망은 특징추출기(Feature Extraction)와 분류기(Classifier)가 합쳐져 있는 
# 구조이다. 특징추출기 역할을 하는 것은 convolution layer와 pooling layer이며, Dense layer는 분류기 역할이다.
# cnn -> pooling -> cnn -> pooling -> flatten -> dense ->dropout -> dense가 한가지 예이다.
# cnn과 pooling은 교차되어 배치되고, 분류기에는 Dense레이어가 배치되며, 과적합을 막기 위해 Dropout레이어가 Dense와
# Dense사이에 배치된다. 마지막 Dense레이어 뒤에는 Dropout레이어가 배치되지 않는다. 하나씩 살펴보자

In [ ]:
# 컨볼루션 레이어

# 컨볼루션 레이어(Convolution Layer)는 말 그대로 컨볼루션 연산을 하는 레이어이다. 여기서 사용되는 필터는 사람이
# 미리 정해놓은 것이 아니라 네트워크의 학습을 통해 자동으로 추출된다. 우리가 코드에서 지정해야 할 값은 필터를 구성
# 하는 각 셀들의 값들이 아니라 필터의 개수 정도이다.
# 컨볼루션 레이어는 1차원부터 3차원까지 다양하게 사용할 수 있지만 여기서는 가장 기본이 되는 2차원으로 설명한다.

# 컨볼루션 레이어는 각 채널에 대해 계산된 갑을 합쳐서 새로운 이미지를 만들어낸다. 이때 새로운 이미지의 마지막 
# 차원 수는 필터의 수와 동일하다. 일반적인 컨볼루션 신경망은 여러 개의 컨볼루션 레이어를 쌓으면서 뒤쪽으로 갈수록
# 필터의 수를 점점 늘리기 때문에 이미지의 마지막 차원 수는 점점 많아지게 된다.

# 2차원 이미지를 다루는 컨볼루션 레이어를 생성하는 코드는 다음과 같다.
# conv1 = tf.keras.layers.Conv2D(kernel_size=(3,3), strides=(2,2), padding='valid', filters=16)
# Conv2D 레이어를 생성할 때의 주요 매개변수들은 kernel_size, strides, padding, filters의 네가지이다.
# kernel_size는 필터행렬의 크기이다. 수용영역(receptive filed)이라고도 하고 (row수, column수)를 나타내고 하나만 
# 쓰면 row수와 column수를 같게 한다는 의미이다.
# strides는 필터가 계산과정에서 한 스텝마다 이동하는 크기이다. Default값은 (1,1)이고 (2,2)로 설정할 경우 한 칸씩
# 건너뛰면서 계산하게 된다. 동일한 조건에서 strides가 달라지면 결과 이미지의 크기에 영향을 준다.
# padding은 컨볼루션 연산 전에 입력 이미지 주변에 빈 값을 넣을지 지정하는 옵션으로서 'valid'와 'same' 두가지중
# 하나를 사용한다. valid는 빈 값을 사용하지 않는 옵션이고 same은 빈 값을 넣어서 출력 이미지의 크기를 입력과 같도록
# 보존하는 옵션이다. 이때 빈 값으로 0이 쓰이는 경우에 제로 패딩(zero padding)이라고 부른다.
# filters는 필터의 개수이다. 필터의 개수는 네트워크가 얼마나 많은 특징을 추출할 수 있는지 결정하기 떄문에 많을수록
# 좋지만 너무 많을 경우 학습 속도가 느려질 수 있고 과적합이 발생할 수도 있다. 가장 유명한 CNN의 하나인 VGG는 
# 네트워크가 깊어질수록 필터의 수를 2배씩 늘려간다. 

In [ ]:
# 풀링 레이어

# 이미지를 구성하는 픽셀중 인접한 픽셀들은 비슷한 정보를 가지고 있는 경우가 많이 있다. 이런 이미지의 크기를 줄이면서
# 중요한 정보만 남기기 위해 subsampling이라는 기법을 사용한다. 메모리의 크기는 제한적이어서 중요 정보만 남기는 과정은
# 효율적인 메모리 사용에 도움이 되고 또 계산할 정보가 줄어들기 때문에 과적합을 방지하는 효과도 있다. 
# 이 과정에서 사용하는 레이어가 Pooling layer이다. Max Pooling layer와 Average Pooling layer가 있다. 
# Conv2D 레이어와 같이 사용되는 MaxPool2D 레이어의 생성코드이다.
# pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))
# pool_size는 한 번에 Max연산을 수행할 범위이다. pool_size=(2,2)이므로 2*2사각형안에서 최댓값만 남기는 연산을 한다.
# strides는 Conv2D레이어에서와 같이 한 스텝마다 이동하는 크기이다.
# pool_size=(2,2)와 strides=(2,2)옵션으로 MaxPool2D레이어를 사용하면 결과 이미지의 크기는 너비, 높이가 각각 절반씩
# 줄어들게 된다. 홀수일 때는 절반에서 내림한 값이 된다.(예:5->2)
# Pooling layer에는 가중치가 존재하지 않기 때문에 학습되지 않으며 네트워크 구조에 따라 생략되기도 한다. 

In [ ]:
# 드롭아웃 레이어

# 네트워크의 과적합을 막기 위한 딥러닝 연구자들의 노력의 결과로 Georffrey Hinton교수팀이 "학습과정에서 무작위로 뉴런
# 의 부분집합을 제거하는 것이 뉴런들간의 공모(conspiracy)를 막고 과적합(overfitting)을 감소시킬 것"이라고 했다.
# 위의 언급대로 드롭아웃 레이어는 학습과정에서 무작위로 뉴런의 부분집합을 제거하는 것이다. 네트워크가 학습할 때 같은 
# 레이어에 있는 뉴런들은 결과값에 의해 서로 같은 영향을 받는다. 따라서 각 뉴런의 계산결과를 모두 더해서 나오는 
# 결과값은 한쪽으로 치우치게 되는데 이를 막기 위한 드롭아웃 레이어는 학습과정에서는 확률적으로 일부 뉴런에 대한 영향을
# 끊고 테스트할 때는 정상적으로 모든 값을 계산한다.
# pool1 = tf.keras.layers.Dropout(rate=0.3) 
# rate는 제외할 뉴런의 비율을 나타낸다. 간단한 레이어지만 AlexNet, VGG, GoogLeNet, DenseNet등 거의 모든 주요한 
# CNN에서 이 드롭아웃레이어를 사용됐다. 드롭아웃 레이어도 가중치가 없기 때문에 학습되지 않는다.

In [ ]:
# 지금까지 CNN의 중요 레이어들에 대해 알아봤다. 다시 정리해보면 Convolution layer는 특징을 추출하는 역할, Pooling
# layer는 중요한 정보만 남기고 계산부담을 줄여주는 역할, Dropout layer는 과적합을 방지하는 역할을 한다.
# 이중에서 학습이 가능한 것은 Convolution layer뿐이다.

##### Fashion MNIST 데이터세트에 CNN적용하기